# Reducción de Ruido con BM3D

**BM3D (Block-Matching and 3D Filtering)** es un algoritmo potente para limpiar ruido de imágenes con resultados muy buenos. No funciona como los filtros normales (tipo desenfoque), sino que se basa en la **similitud estadística** que existe en las imágenes reales.

El algoritmo tiene dos partes principales:

1. **Emparejamiento de Bloques**: Las imágenes naturales tienen muchas zonas que se parecen. BM3D busca esos bloques similares en toda la imagen.

2. **Filtrado 3D**: Junta los bloques parecidos en un arreglo tridimensional y los filtra juntos. De esta forma se quita ruido de varios bloques al mismo tiempo, aprovechando que las imágenes tienen partes repetidas.

- **Mantiene los detalles**: Los bordes y detalles finos se conservan mucho mejor que con otros filtros
- **Se adapta a la imagen**: Ajusta su comportamiento según lo que detecta
- **Lo puedes controlar**: El parámetro sigma te deja elegir cuánto limpiar vs cuánto detalle mantener
- **Diferentes modos**: Hay varios perfiles para diferentes situaciones, desde rápido hasta máxima calidad

### Cómo se Usa en VCF

Dentro del framework **VCF (Video Compression Framework)**, BM3D entra en la **fase de decodificación** como **filtro posterior**:

```
Imagen Original → Codificación → Almacenamiento/Transmisión → Decodificación → BM3D → Imagen Final
```

**Lo que consigues en VCF:**
- Elimina los artefactos que deja la compresión
- Mejora cómo se ve la imagen sin comprometer la compresión
- Puedes balancear velocidad vs calidad según el perfil
- Funciona con diferentes transformadas espaciales (DCT, DWT, etc.)

## Integrantes del Proyecto

- **Isabel Pelaya Galindo Ibáñez**
- **Esther Ibáñez Mingorance**
- **José Luis López García**
- **Juan Rafael Madolell Usero**

In [ ]:
import subprocess
import sys

packages = ['matplotlib', 'scipy', 'bm3d']
for package in packages:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', package], check=False)

In [ ]:
%%writefile ../src/BM3D.py
'''Eliminación de ruido en imágenes utilizando Block-matching y filtrado 3D (BM3D).
Solo es efectivo durante la decodificación.

Este módulo implementa una interfaz de BM3D para el framework VCF, soportando tanto imágenes
en escala de grises como en color. BM3D es un potente algoritmo de denoising que utiliza
filtrado colaborativo en el dominio de la transformada 3D.
'''

import numpy as np
import logging
import os

import tempfile
import builtins

# Create a valid temporary file path
temp_desc_path = os.path.join(tempfile.gettempdir(), "description.txt")

# Write the description to the valid temporary file
with open(temp_desc_path, 'w', encoding='utf-8') as f:
    f.write(__doc__)

# Monkeypatch open to redirect /tmp/description.txt to our valid temp file
# This is necessary because parser.py (which we cannot edit) hardcodes /tmp/description.txt
_original_open = builtins.open

def _redirect_open(file, *args, **kwargs):
    if file == "/tmp/description.txt":
        return _original_open(temp_desc_path, *args, **kwargs)
    return _original_open(file, *args, **kwargs)

builtins.open = _redirect_open

try:
    import parser
finally:
    # Restore original open
    builtins.open = _original_open
import main
import importlib
import cv2

try:
    import bm3d
except ImportError:
    logging.error("El paquete 'bm3d' es necesario para este módulo. Instálalo con 'pip install bm3d'.")
    raise

# Parámetros por defecto de BM3D
default_sigma = 10.0
default_profile = 'np' # Perfil Normal

# Parámetros del parser para BM3D en la decodificación
# sigma_bm3d: Desviación estándar del ruido. A mayor valor, más limpieza pero más desenfoque.
# psd_bm3d: Ruta a un archivo .npy con la densidad espectral de potencia (para ruido correlacionado).
# psf_bm3d: Ruta a un archivo .npy con la función de dispersión de punto (para deblurring).
# profile_bm3d: Perfil de rendimiento: np (normal), lc (baja complejidad), high (alta calidad), vn (ruido extremo).
parser.parser_decode.add_argument("-s_bm3d", "--sigma_bm3d", type=float, 
                                 help=f"Noise standard deviation (sigma) for BM3D. Higher values remove more noise but may blur details (default: {default_sigma})", 
                                 default=default_sigma)
parser.parser_decode.add_argument("-psd_bm3d", "--psd_bm3d", type=str, 
                                 help="Path to a .npy file containing the noise Power Spectral Density (PSD) for correlated noise.", 
                                 default=None)
parser.parser_decode.add_argument("-h_bm3d", "--psf_bm3d", type=str, 
                                 help="Path to a .npy file containing the Point Spread Function (PSF) for deblurring. If provided, BM3D deblurring is performed.", 
                                 default=None)
parser.parser_decode.add_argument("-p_bm3d", "--profile_bm3d", type=str, 
                                 choices=['np', 'lc', 'high', 'vn'],
                                 help=f"BM3D profile: np (normal), lc (low complexity), high (high quality), vn (very noisy) (default: {default_profile})", 
                                 default=default_profile)

import no_filter

# Parsear argumentos para obtener las opciones elegidas
args = parser.parser.parse_known_args()[0]

class CoDec(no_filter.CoDec):
    """
    Codec basado en BM3D para la eliminación de ruido.
    Hereda de no_filter.CoDec, que gestiona el flujo de descompresión básico.
    """

    def __init__(self, args):
        logging.debug(f"Inicializando CoDec BM3D con argumentos: {args}")
        super().__init__(args)
        self.args = args

    def encode(self):
        """
        Método de codificación con corrección de argumentos.
        Necesario porque la clase base usa defaults incorrectos para este entorno.
        """
        img = self.encode_read(self.args.original)
        compressed_img = self.compress(img)
        output_size = self.encode_write(compressed_img, self.args.encoded)
        return output_size

    def decode(self):
        """
        Método de decodificación sobrescrito para aplicar el filtro tras la descompresión.
        """
        # Leer los datos comprimidos
        compressed_k = self.decode_read(self.args.encoded)
        
        # Descomprimir los datos (normalmente llama a los decodificadores de transformación espacial/color)
        k = self.decompress(compressed_k)
        logging.info(f"Forma de la imagen descomprimida: {k.shape}, tipo de dato: {k.dtype}")
        
        # Aplicar el filtro BM3D
        y = self.filter(k)
        
        # Escribir la imagen de salida
        output_size = self.decode_write(y, self.args.decoded)
        return output_size
            
    def filter(self, img):
        """
        Aplica el filtrado BM3D (denoising/deblurring) a la imagen.
        Soporta imágenes monocromáticas, RGB y multicanal.
        """
        logging.info(f"Aplicando filtro BM3D (sigma_bm3d={self.args.sigma_bm3d}, perfil='{self.args.profile_bm3d}')")
        
        # Validación de parámetros
        if self.args.sigma_bm3d < 0:
            logging.warning(f"Sigma inválido ({self.args.sigma_bm3d}). Ajustando a 0.")
            self.args.sigma_bm3d = 0.0

        # Normalizar la imagen al rango [0, 1] para el procesamiento BM3D
        img_float = img.astype(np.float32) / 255.0
        sigma_normalized = self.args.sigma_bm3d / 255.0
        
        # Manejar PSD para ruido correlacionado
        if self.args.psd_bm3d and os.path.exists(self.args.psd_bm3d):
            try:
                logging.info(f"Cargando PSD de ruido desde {self.args.psd_bm3d}")
                noise_spec = np.load(self.args.psd_bm3d)
            except Exception as e:
                logging.error(f"Error al cargar PSD: {e}. Usando sigma como respaldo.")
                noise_spec = sigma_normalized
        else:
            noise_spec = sigma_normalized

        # Cargar PSF si se solicita deblurring
        psf = None
        if self.args.psf_bm3d and os.path.exists(self.args.psf_bm3d):
            try:
                logging.info(f"Cargando PSF desde {self.args.psf_bm3d} para deblurring")
                psf = np.load(self.args.psf_bm3d)
            except Exception as e:
                logging.error(f"Error al cargar PSF: {e}. Solo se realizará denoising.")

        # Seleccionar mapeo de perfil
        profile_map = {
            'np': bm3d.BM3DProfile(),
            'lc': bm3d.BM3DProfileLC(),
            'high': bm3d.BM3DProfileHigh(),
            'vn': bm3d.BM3DProfileVN()
        }
        selected_profile = profile_map.get(self.args.profile_bm3d, bm3d.BM3DProfile())
        
        try:
            # Verificar si la imagen es en color (3 canales) o escala de grises
            if len(img.shape) == 3:
                channels = img.shape[2]
                if channels == 3:
                    if psf is not None:
                        logging.info("Aplicando deblurring BM3D a imagen RGB (canal por canal).")
                        denoised = np.zeros_like(img_float)
                        for i in range(3):
                            denoised[:,:,i] = bm3d.bm3d_deblurring(img_float[:,:,i], noise_spec, psf, profile=selected_profile)
                    else:
                        logging.info("Entrada reconocida como RGB. Usando bm3d_rgb.")
                        denoised = bm3d.bm3d_rgb(img_float, noise_spec, profile=selected_profile)
                else:
                    logging.info(f"Entrada reconocida como imagen de {channels} canales. Aplicando BM3D por canal.")
                    denoised = np.zeros_like(img_float)
                    for i in range(channels):
                        if psf is not None:
                             denoised[:,:,i] = bm3d.bm3d_deblurring(img_float[:,:,i], noise_spec, psf, profile=selected_profile)
                        else:
                             denoised[:,:,i] = bm3d.bm3d(img_float[:,:,i], noise_spec, profile=selected_profile)
            else:
                if psf is not None:
                    logging.info("Aplicando deblurring BM3D a imagen en escala de grises.")
                    denoised = bm3d.bm3d_deblurring(img_float, noise_spec, psf, profile=selected_profile)
                else:
                    logging.info("Entrada reconocida como escala de grises. Usando bm3d.")
                    denoised = bm3d.bm3d(img_float, noise_spec, profile=selected_profile)
            
            # Recortar y convertir de nuevo a uint8 [0, 255]
            result = (np.clip(denoised, 0, 1) * 255).astype(np.uint8)
            return result
        except Exception as e:
            logging.error(f"Error durante el procesamiento BM3D: {e}")
            logging.warning("Volviendo a la imagen original debido a un error.")
            return img

if __name__ == "__main__":
    main.main(parser.parser, logging, CoDec)


In [ ]:
%%writefile ../src/visualization.py
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np

def show_images(original_path, encoded_path, decoded_path, title):
    """
    Función para visualizar y comparar las imágenes original, codificada y decodificada.
    
    Parámetros:
        original_path: ruta a la imagen original
        encoded_path: ruta a la imagen codificada (coeficientes)
        decoded_path: ruta a la imagen decodificada
        title: título para la visualización
    """
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle(title, fontsize=16)
    
    # Mostrar imagen original
    if os.path.exists(original_path):
        img = cv2.cvtColor(cv2.imread(original_path), cv2.COLOR_BGR2RGB)
        axs[0].imshow(img)
        axs[0].set_title(f'Original\nTamaño: {os.path.getsize(original_path)/1024:.1f} KB')
    else:
        axs[0].text(0.5, 0.5, 'No encontrada', ha='center')
    axs[0].axis('off')
    
    # Mostrar imagen codificada (Coeficientes)
    # Nota: Se muestra el archivo de coeficientes como imagen. Puede parecer ruido o bloques.
    if os.path.exists(encoded_path):
        try:
            # Intentar leer con OpenCV (maneja archivos TIF)
            img_enc = cv2.imread(encoded_path, cv2.IMREAD_UNCHANGED)
            if img_enc is not None:
                if len(img_enc.shape) == 3:
                     # Convertir BGR a RGB si es imagen de color
                     img_enc = cv2.cvtColor(img_enc, cv2.COLOR_BGR2RGB)
                # Mostrar los coeficientes sin normalización
                axs[1].imshow(img_enc, cmap='gray')
            else:
                 axs[1].text(0.5, 0.5, 'No se pudo leer TIF', ha='center')
        except Exception as e:
             axs[1].text(0.5, 0.5, f'Error: {e}', ha='center')
        
        # Calcular y mostrar tamaños de archivos
        weights_path = encoded_path.replace('.tif', '_weights.bin')
        w_size = os.path.getsize(weights_path) if os.path.exists(weights_path) else 0
        enc_size = os.path.getsize(encoded_path)
        total_size = enc_size + w_size
        axs[1].set_title(f'Coeficientes + Pesos\nTIF: {enc_size/1024:.1f} KB | Pesos: {w_size/1024:.1f} KB\nTotal: {total_size/1024:.1f} KB')
    else:
        axs[1].text(0.5, 0.5, 'No encontrada', ha='center')
    axs[1].axis('off')
    
    # Mostrar imagen decodificada (reconstruida)
    if os.path.exists(decoded_path):
        img_dec = cv2.cvtColor(cv2.imread(decoded_path), cv2.COLOR_BGR2RGB)
        axs[2].imshow(img_dec)
        axs[2].set_title(f'Decodificada\nTamaño: {os.path.getsize(decoded_path)/1024:.1f} KB')
    else:
        axs[2].text(0.5, 0.5, 'No encontrada', ha='center')
    axs[2].axis('off')
    
    plt.tight_layout()
    plt.show()

def load_rgb(path):
    """Carga una imagen en formato RGB."""
    if not os.path.exists(path): return np.zeros((10,10,3), dtype=np.uint8)
    img = cv2.imread(path)
    if img is None: return np.zeros((10,10,3), dtype=np.uint8)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def show_results_bm3d(img_name, original_path, filtered_paths):
    """Muestra los resultados: Original + 3 filtradas (BM3D)."""
    # Configuración de niveles
    levels = ['Baja', 'Media', 'Alta']
    
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    
    # Imagen Original
    axes[0].imshow(load_rgb(original_path))
    axes[0].set_title(f"Original ({img_name})", fontsize=12, fontweight='bold')
    axes[0].axis('on')
    axes[0].set_xticks([]); axes[0].set_yticks([])
    
    # Resultados Filtrados (BM3D)
    for i in range(3):
        if i < len(filtered_paths):
            axes[i+1].imshow(load_rgb(filtered_paths[i]))
            axes[i+1].set_title(f"BM3D - {levels[i]}", fontsize=12)
            axes[i+1].axis('on')
            axes[i+1].set_xticks([]); axes[i+1].set_yticks([])
        
    plt.tight_layout()
    plt.suptitle(f"Resultados: {img_name}", fontsize=16, y=1.05)
    plt.show()

def show_results_nlm(img_name, original_path, filtered_paths):
    """Muestra los resultados: Original + 3 filtradas (NLM)."""
    # Configuración de niveles
    levels = ['Baja', 'Media', 'Alta']
    
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    
    # Imagen Original
    axes[0].imshow(load_rgb(original_path))
    axes[0].set_title(f"Original ({img_name})", fontsize=12, fontweight='bold')
    axes[0].axis('on')
    axes[0].set_xticks([]); axes[0].set_yticks([])
    
    # Resultados Filtrados (NLM)
    for i in range(3):
        if i < len(filtered_paths):
            axes[i+1].imshow(load_rgb(filtered_paths[i]))
            axes[i+1].set_title(f"NLM - {levels[i]}", fontsize=12)
            axes[i+1].axis('on')
            axes[i+1].set_xticks([]); axes[i+1].set_yticks([])
        
    plt.tight_layout()
    plt.suptitle(f"Resultados: {img_name}", fontsize=16, y=1.05)
    plt.show()

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import display, Image
import sys
sys.path.append('../src')
from visualization import show_results_bm3d

In [ ]:
# Crear la carpeta /img si no existe
import os
img_dir = '../img'
if not os.path.exists(img_dir):
    os.makedirs(img_dir)
    print(f"Carpeta '{img_dir}' creada exitosamente")
else:
    print(f"La carpeta '{img_dir}' ya existe")

## Parametros BM3D
BM3D agrupa bloques parecidos en 3D y los filtra todos juntos para limpiar la imagen manteniendo mucha calidad.

Configuración:
- `-s_bm3d` o `--sigma_bm3d`: **Nivel de ruido (Sigma)**. Es el parámetro principal de intensidad.
- `-p_bm3d` o `--profile_bm3d`: **Perfil**. Permite elegir entre:
    - `np`: Perfil Normal (predeterminado).
    - `lc`: Baja Complejidad (más rápido).
    - `high`: Alta Calidad (más lento).
    - `vn`: Ruido Extremo (para varianzas muy altas).
- `-psd_bm3d` o `--psd_bm3d`: **PSD (para ruido específico)**. Permite pasar un archivo `.npy` si el ruido no es blanco (ruido correlacionado).
- `-h_bm3d` o `--psf_bm3d`: **PSF (si hay desenfoque)**. Si se proporciona un archivo `.npy`, el algoritmo aplica **deblurring** junto con el denoising.

--- 
## Codificar imagen Pajaro.png

In [ ]:
# Codificar imagen original una sola vez
!python ../src/BM3D.py encode -o ../tmp/original.png -e ../img/Pajaro_encoded

## Comparación: Variando el Parámetro Sigma

En esta prueba cambiamos el valor de **sigma**, que controla la cantidad de limpieza de ruido:

- **Sigma = 5**: El filtro es muy suave. Deja casi todo como está, limpia poco ruido. Útil si la imagen tiene poco ruido o quieres conservar todos los detalles.

- **Sigma = 15**: Es el punto intermedio. Limpia más ruido pero sigue respetando los detalles de la imagen. La mayoría de las veces es un buen balance.

- **Sigma = 30**: Aquí sí limpia bastante. El ruido casi desaparece pero los detalles pueden quedar un poco suavizados. Es para cuando el ruido es realmente fuerte.

La idea es que veas qué pasa cuando cambias este parámetro: cómo se mejora la limpieza pero también cómo se pueden perder detalles si lo pones demasiado alto.

In [ ]:
# Decodificar con diferentes valores de sigma
!python ../src/BM3D.py decode -e ../img/Pajaro_encoded -d ../img/pajaro_bm3d_sigma5.png --sigma_bm3d 5.0
!python ../src/BM3D.py decode -e ../img/Pajaro_encoded -d ../img/pajaro_bm3d_sigma15.png --sigma_bm3d 15.0
!python ../src/BM3D.py decode -e ../img/Pajaro_encoded -d ../img/pajaro_bm3d_sigma30.png --sigma_bm3d 30.0
show_results_bm3d('Pajaro.png (Sigma)', '../docs/Pajaro.png', ['../img/pajaro_bm3d_sigma5.png', '../img/pajaro_bm3d_sigma15.png', '../img/pajaro_bm3d_sigma30.png'])

## Comparación: Variando el Perfil

El **perfil** es lo que controla la estrategia de filtrado. Hay varias opciones dependiendo de para qué quieras usarlo:

- **Numeración (np)**: Es rápido pero menos preciso. Usa menos bloques para emparejar, así que funciona más rápido. Bueno si necesitas velocidad.

- **Lossless (lc)**: Intenta que casi no se pierda nada de la imagen original. Más lento pero los resultados son muy similares a lo original.

- **High Quality (high)**: Va directo a la máxima calidad posible. Usa las mejores opciones disponibles. Mucho más lento pero el resultado es el mejor que puedes conseguir.

Lo que ves aquí es exactamente eso: el mismo parámetro sigma pero usando diferentes estrategias. Puedes observar cómo cambia la velocidad y la precisión según elijas.

In [ ]:
# Decodificar con diferentes perfiles de BM3D
!python ../src/BM3D.py decode -e ../img/Pajaro_encoded -d ../img/pajaro_bm3d_np.png --sigma_bm3d 15.0 --profile_bm3d np
!python ../src/BM3D.py decode -e ../img/Pajaro_encoded -d ../img/pajaro_bm3d_lc.png --sigma_bm3d 15.0 --profile_bm3d lc
!python ../src/BM3D.py decode -e ../img/Pajaro_encoded -d ../img/pajaro_bm3d_high.png --sigma_bm3d 15.0 --profile_bm3d high
show_results_bm3d('Pajaro.png (Perfiles)', '../docs/Pajaro.png', ['../img/pajaro_bm3d_np.png', '../img/pajaro_bm3d_lc.png', '../img/pajaro_bm3d_high.png'])